In [1]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
dataType='val2017'
annFile=f'dataset/annotations/instances_{dataType}.json'
coco=COCO(annFile)

loading annotations into memory...
Done (t=1.75s)
creating index...
index created!


In [3]:
catNms = ['airplane', 'cat', 'dog']
catIds = coco.getCatIds(catNms=catNms)
cat_dict = dict(zip(catIds,catNms))

In [4]:
cat_ids = {}
x_id = []
Y = []
total = 0
for cat in catIds:
  cat_ids[cat_dict[cat]] = coco.getImgIds(catIds=cat)
  for img_id in cat_ids[cat_dict[cat]]:
        x_id.append(img_id)
        Y.append(cat_dict[cat])
  print(cat_dict[cat], len(cat_ids[cat_dict[cat]]))
  total += len(cat_ids[cat_dict[cat]])
print(f'\ntotal = {len(catIds)}\ntotal no. of imgs = {total}')

airplane 97
cat 184
dog 177

total = 3
total no. of imgs = 458


In [5]:
from skimage.transform import resize as imresize
import skimage.io as io
import os

def load_images(img_ids, img_dir, img_size, preprocessor):
    x = []
    for img_id in img_ids:
        img_file_name = coco.loadImgs(img_id)[0]['file_name']
        img_complete_path = os.path.join(img_dir,img_file_name)
        img = io.imread(img_complete_path)
        img = imresize(img, (img_size,img_size,3), mode='reflect')
#         img = preprocessor(img)
        x.append(img)
    return x

In [8]:
import tensorflow.keras as keras
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

# from tensorflow.keras.applications.imagenet_utils import decode_predictions
#
# img_dir = 'dataset/val2017/'
# img_size = 299
# preprocessor = inception_v3.preprocess_input
# X = load_images(x_id, img_dir, img_size, preprocessor)
# X = np.array(X)
# print(X.shape)

In [ ]:
# with open('images.pkl', 'wb') as f:
#     pickle.dump(X,f)

In [9]:
with open('images.pkl', 'rb') as f:
    X = pickle.load(f)

In [78]:
img1 = keras.layers.Input(shape=(299,299,3))
img2 = keras.layers.Input(shape=(299,299,3))
merge_layer = keras.layers.Lambda(lambda img1: (base_model(img1[0]) - base_model(img1[1])) ** 2)([img1, img2])

In [87]:
### loading the models
inception_model = inception_v3.InceptionV3(weights="imagenet", include_top=True)
base_model = Model(inception_model.input, inception_model.get_layer("avg_pool").output)
del inception_model

### transition
img1 = keras.layers.Input(shape=(299,299,3))
img2 = keras.layers.Input(shape=(299,299,3))
merge_layer = keras.layers.Lambda(lambda img: (base_model(img[0]) - base_model(img[1])) ** 2)([img1, img2])
# vec1 = base_model(img1)
# vec2 = base_model(img2)
# merge_layer = keras.layers.Lambda(lambda vec: (vec[0] - vec[1]) ** 2)([vec1, vec2])

full_model = keras.Model([img1, img2], similarity_model(merge_layer))

In [89]:
vec1 = keras.layers.Input(shape=(2048,))
vec2 = keras.layers.Input(shape=(2048,))
merge_layer_vec = keras.layers.Lambda(lambda vec: (vec[0] - vec[1]) ** 2)([vec1, vec2])

vec_similarity_model = keras.Model([vec1,vec2], similarity_model(merge_layer_vec))

In [ ]:
vec_similarity_model.summary()

In [83]:
full_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
model_10 (Model)                (None, 2048)         21802784    input_31[0][0]                   
                                                                 input_32[0][0]                   
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, 2048)         0           model_10[1][0]                   
          

In [88]:
full_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
input_35 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 2048)         0           input_34[0][0]                   
                                                                 input_35[0][0]                   
__________________________________________________________________________________________________
model (Model)                   (None, 1)            2368833     lambda_10[0][0]                  
Total para

In [35]:
# similarity_model_out = similarity_model(base_model.layers[-1].output)
# full_model2 = keras.Model(base_model.input, similarity_model_out)

In [41]:
import pickle
with open('exp1.pkl', 'rb') as f:
    exps1 = pickle.load(f)
with open('exp2.pkl', 'rb') as f:
    exps2 = pickle.load(f)

In [43]:
no_superpixels = 5
def get_imgs_from_exps(exps):
    return np.array([exp.get_image_and_mask(exp.top_labels[0], positive_only=True,
                                            num_features=no_superpixels, hide_rest=True)[0]
                     for exp in exps])

In [44]:
%time imgs1 = get_imgs_from_exps(exps1)
%time imgs2 = get_imgs_from_exps(exps2)

CPU times: user 1h 3min 20s, sys: 12min 25s, total: 1h 15min 46s
Wall time: 3min 47s
CPU times: user 1h 3min 34s, sys: 13min 8s, total: 1h 16min 43s
Wall time: 3min 57s


In [45]:
from metrics import *

In [46]:
calc_identity(imgs1,imgs2)

true:  103 wrong:  355 total:  458


(77.51091703056768, 103, 458)

In [47]:
calc_separability(imgs1)

true:  209304 wrong:  2 total:  209306


(2, 458, 209306, 0.0009555387805414082)

In [52]:
## vectorizer to improve clustering performance
vectors = base_model.predict(X)
with open('vectorized_img', 'wb') as f:
    pickle.dump(vectors,f)

In [85]:
vectors.shape

(458, 2048)

In [113]:
### making the distance matrix
eps = 1e-6
distance_matrix = np.zeros((vectors.shape[0], vectors.shape[0]))
for i, vec1 in enumerate(vectors):
    for j, vec2 in enumerate(vectors):
        distance_matrix[i][j] = 1 /( vec_similarity_model.predict([vec1[None,:],vec2[None,:]]) + eps)if i != j else 1
    if i % 50 == 0:
        print(f"done with {i} instances")

In [141]:
from sklearn.cluster import AgglomerativeClustering
cluster_model = AgglomerativeClustering(n_clusters=20, affinity='precomputed', linkage='complete')
cluster_model.fit(distance_matrix)

AgglomerativeClustering(affinity='precomputed', compute_full_tree='auto',
            connectivity=None, linkage='complete', memory=None,
            n_clusters=20, pooling_func='deprecated')

In [142]:
clusters = np.unique(cluster_model.labels_)
cluster = {}
for i in clusters:
    cluster[i] = np.where(cluster_model.labels_ == i)[0]
    print(f"cluster {i} has {len(cluster[i])} instances")
    
# np.array(Y)[cluster[16]]

cluster 0 has 46 instances
cluster 1 has 6 instances
cluster 2 has 39 instances
cluster 3 has 48 instances
cluster 4 has 17 instances
cluster 5 has 11 instances
cluster 6 has 4 instances
cluster 7 has 52 instances
cluster 8 has 6 instances
cluster 9 has 8 instances
cluster 10 has 1 instances
cluster 11 has 13 instances
cluster 12 has 42 instances
cluster 13 has 33 instances
cluster 14 has 42 instances
cluster 15 has 6 instances
cluster 16 has 50 instances
cluster 17 has 10 instances
cluster 18 has 22 instances
cluster 19 has 2 instances


In [174]:
imgs1.shape

(458, 299, 299, 3)

In [178]:
score = 0
for cluster_id, imgs_id in cluster.items():
    if len(imgs_id) < 2:
        continue
    cluster_imgs = imgs1[imgs_id]
    for img1 in cluster_imgs:
        for img2 in cluster_imgs:
            score += full_model.predict([img1[None,:],img2[None,:]])
        if i%5 == 0:
            print(f"done with {i} imgs")
    print(f"cluster {cluster_id} done\nscore = {score}\n________________________________________")
score /= (458**2)

cluster 0 done
________________________________________
cluster 1 done
________________________________________
cluster 2 done
________________________________________
cluster 3 done
________________________________________
cluster 4 done
________________________________________
cluster 5 done
________________________________________
cluster 6 done
________________________________________
cluster 7 done
________________________________________
cluster 8 done
________________________________________
cluster 9 done
________________________________________
cluster 11 done
________________________________________
cluster 12 done
________________________________________
cluster 13 done
________________________________________
cluster 14 done
________________________________________
cluster 15 done
________________________________________
cluster 16 done
________________________________________
cluster 17 done
________________________________________
cluster 18 done
_________________________

In [143]:
# from sklearn.cluster import SpectralClustering
# cluster_model = SpectralClustering(n_clusters=15, affinity='precomputed', n_init=1, assign_labels='discretize')
# cluster_model.fit(distance_matrix)

In [179]:
score

array([[0.05729971]], dtype=float32)